In [1]:
import wikipedia
import os
import json
import tqdm

import multiqa_utils.general_utils as gu
import multiqa_utils.qampari_utils as qu
import multiqa_utils.retrieval_utils as ru
import multiqa_utils.wikipedia_utils as wu

%load_ext autoreload
%autoreload 2

In [2]:
## Lets do it!

In [3]:
tagme_wiki_dir = "/scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/"

In [4]:
postp_tagme_wiki_dir = f"{tagme_wiki_dir}postprocessed/"

In [5]:
all_titles_path = f"{postp_tagme_wiki_dir}wikiversion_title_set.json"

In [6]:
gpt_ans_raw_path = "/scratch/ddr8143/multiqa/qampari_data/qmp_simple_gpt3_answers.json"
gpt_ans_path = "/scratch/ddr8143/multiqa/qampari_data/qmp_simple_gpt3_answers_structured.json"

In [7]:
curr_cache = json.load(open('/scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/str2wikipage_cache.json'))

In [8]:
ru.aggregate_strs_to_add_to_cache(
    gpt_ans_path=None,
    elq_ans_path=None,
    processed_wikitags_path_regexp=None,
    #output_path=None,
    use_tqdm=True,
    curr_cache=curr_cache,
)

>> Load existing string list: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/strs_to_add_to_cache_v0.json
>> Initial string list length: 3988891
>> Removing strings already in cache
>> New string list length: 3988715
>> Writing file
>> Dumped to: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/strs_to_add_to_cache_v0.json


In [9]:
strs_to_add = json.load(open("/scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/strs_to_add_to_cache_v0.json"))

In [ ]:
wu.build_str2wikipage_cache(
    strs_to_add=strs_to_add,
    force=False,
    use_tqdm=True,
    write_every=10000,
    #disambig_cache_path=default,
    #cache_path=default,
    #wikiversion_title_set_path=default,
)

>> Loading the cache
>> Initial cache size: 6308282
>> Adding new strings to cache: 3988715


  0%|                                                                                                                                                                                                 | 25/3988715 [00:13<579:55:25,  1.91it/s]/ext3/miniconda3/envs/multiqa/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /ext3/miniconda3/envs/multiqa/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
  0%|                                                                            